# Computational Experiments on Maass Forms

In [1]:
T = matrix([[1, 1], [0, 1]])
S = matrix([[0, -1], [1, 0]])
Id = matrix([[1, 0], [0, 1]])

def act(gamma, z):
    """
    Computes the action of matrix gamma on point z
    """
    a, b = gamma[0]
    c, d = gamma[1]
    return (a*z + b)/ (c*z + d)

def is_in_fund_domain(z):
    """
    True if z is in fundamental domain, False otherwise.
    """
    x = z.real()
    if x < -1/2 or x > 1/2:
        return False
    if z.norm() < 1:
        return False
    return True


def pullback(z):
    """
    Returns matrix gamma, point z^*, where gamma z = z^* and z^* is in the fundamental domain
    """
    gamma = Id
    while not is_in_fund_domain(z):
        x, y = z.real(), z.imag()
        xshift = -floor(x + .5)
        gamma = T**xshift * gamma
        z = x + xshift + i*y
        if z.norm() < 1:
            z = act(S, z)
            gamma = S * gamma
    return gamma, z

In [2]:
pullback(CC(3.75 + .25*I))

(
[-2  7]                    
[ 1 -4], 2.00000000000000*I
)

In [3]:
pullback(CDF(3.75 + .25*I))

(
[-2  7]                                               
[ 1 -4], -4.440892098500626e-16 + 1.9999999999999996*I
)

In [4]:
bessel_K(CC(1.0), CC(3.0))

0.0401564311281942

In [5]:
bessel_K(CDF(1.0), CDF(3.0))

0.040156431128194184

In [6]:
def kappa(r, u):
    return (exp(CC(pi) * CC(r) / 2) * bessel_K(CC(i*r), CC(u)))
#def kappa(r, u):
#    return bessel_K(CDF(i*r), CDF(u))

In [7]:
def make_zj_list(Y, Q=50):
    """
    Returns list of z_j = x_j + iY
    """
    ret = []
    for j in range(1-Q, Q+1):
        ret.append((1/(2*Q)*(j - 0.5) + i*Y))
    return ret

def make_zjstar_list(zjlist):
    """
    Returns list of z_j^* in same order as zjlist
    """
    return list(map(lambda zj: pullback(zj)[1], zjlist))

In [8]:
test_zj_list = make_zj_list(0.5, Q=4)
for elem in test_zj_list:
    print(elem)

-0.437500000000000 + 0.500000000000000*I
-0.312500000000000 + 0.500000000000000*I
-0.187500000000000 + 0.500000000000000*I
-0.0625000000000000 + 0.500000000000000*I
0.0625000000000000 + 0.500000000000000*I
0.187500000000000 + 0.500000000000000*I
0.312500000000000 + 0.500000000000000*I
0.437500000000000 + 0.500000000000000*I


In [9]:
test_zjstar_list = make_zjstar_list(test_zj_list)
for elem in test_zjstar_list:
    print(elem)

-0.00884955752212391 + 1.13274336283186*I
-0.101123595505618 + 1.43820224719101*I
-0.342465753424658 + 1.75342465753425*I
0.246153846153846 + 1.96923076923077*I
-0.246153846153846 + 1.96923076923077*I
0.342465753424658 + 1.75342465753425*I
0.101123595505618 + 1.43820224719101*I
0.00884955752212391 + 1.13274336283186*I


In [10]:
def perform_sanity_check_zjstar(zjlist, zjstarlist):
    for zj, zjstar in zip(zjlist, zjstarlist):
        mat, _ = pullback(zj)
        assert (act(mat, CC(zj)) - zjstar).norm() < 0.00000001
    print("Passed.")
perform_sanity_check_zjstar(test_zj_list, test_zjstar_list)

Passed.


In [11]:
def V(n, l, zjlist, r, Q, zjstarlist=None):
    """
    Return the constant V(n, l) --- which also depends on zj, r, and Q --- as
    defined above.
    """
    if zjstarlist is None:
        zjstarlist = make_zjstar_list(zjlist)
    ret = CDF(0)
    assert len(zjstarlist) == 2*Q
    for zj, zjs in zip(zjlist, zjstarlist):
        xj = zj.real()
        xjs = zjs.real()
        yjs = zjs.imag()
        ret += (sqrt(yjs)) * kappa(r, 2 * pi * abs(l) * yjs) * (exp(2*pi*i*(l * xjs - n * xj)))
    ret = ret / (2*Q)
    return CDF(ret)

## Large Speedup TODO

Essentially all computation time is spent currently computing $V(n, l)$ in order to make the matrix. There are $Q^2$ matrix entries, each formed from a sum of $Q$ values. And right now, I'm computing all $Q^3$ K-Bessel functions. But this is excessive. In fact, there are at most $Q^2$ K-Bessel functions that need to be computed:
$$ K_{ir}(2 \pi \lvert \ell \rvert y_j^*),$$
where $1 \leq \lvert \ell \rvert \leq Q$ and $j \in [-Q, Q]$.

Rewriting the matrix-generation code to take advantage of this with caching would make it a $Q^2$ operation (assuming that the Bessel functions are where all of the computation time is being spent).

Section 3.4 of [this paper](http://www2.math.uu.se/~astrombe/papers/kbessel.pdf) gives a further improvement for repeated honing in, as only the values of $r$ are changing. Sections 3.2 and 3.3 might give improvements for an individual run --- I don't quite understand the implied algorithms.

The matrix-generation code should also be vectorized as much as possible.

In [12]:
V(3, 1, test_zj_list, 9.5**.5, 4)

-0.0016458022812606853 + 1.6263032587282567e-19*I

In [13]:
import time

In [14]:
now = time.time()
print(V(3, 1, test_zj_list, 9.5**.5, 4))
print("Took {}".format(time.time() - now))

-0.0016458022812606853 + 1.6263032587282567e-19*I
Took 0.04369521141052246


In [15]:
def make_matrix(size, Y, r, withtime=True):
    if withtime:
        now = time.time()
    Q = size
    zjlist = make_zj_list(Y, Q)
    zjstarlist = make_zjstar_list(zjlist)
    if withtime:
        print("lists made in {}".format(time.time() - now))
    matrix = []
    B = []
    for n in range(-Q, Q+1):
        if withtime:
            now = time.time()
        if n == 0:
            continue
        row = []
        for l in range(-Q, Q+1):
            if l == 0:
                continue
            entry = CDF(V(n, l, zjlist, CDF(r), CDF(Q), zjstarlist=zjstarlist))
            if n == l:
                #entry = entry - sqrt(Y) * bessel_K(i*r, 2 * pi * abs(n) * Y)
                entry = entry - CDF(sqrt(Y)) * kappa(r, 2 * pi * abs(n) * Y)
            if l != 1:
                row.append(entry.n())
            else:
                B.append([-entry.n()])
                #if n == 1:
                #    print(V(n, l, zjlist, r, Q, zjstarlist=zjstarlist))
                #    print(sqrt(Y) * kappa(r, 2 * pi * abs(n) * Y))
        matrix.append(row)
        if withtime:
            print("row {} computed in {} seconds".format(n, time.time() - now))
    return matrix, B

In [16]:
Q = 8
Y = 0.28
r = 9.53369526135
now = time.time()
#testmat, testB = make_matrix(20, 0.5, 3)
testmat, testB = make_matrix(Q, Y, r)
print("Done in {}".format(time.time() - now))

lists made in 0.030083179473876953
row -8 computed in 0.6715025901794434 seconds
row -7 computed in 0.8078534603118896 seconds
row -6 computed in 0.9051103591918945 seconds
row -5 computed in 0.6376907825469971 seconds
row -4 computed in 0.6817789077758789 seconds
row -3 computed in 0.678154468536377 seconds
row -2 computed in 0.6640219688415527 seconds
row -1 computed in 0.6612107753753662 seconds
row 1 computed in 0.6689596176147461 seconds
row 2 computed in 0.8570430278778076 seconds
row 3 computed in 0.9631009101867676 seconds
row 4 computed in 0.7542071342468262 seconds
row 5 computed in 0.7352504730224609 seconds
row 6 computed in 0.7185895442962646 seconds
row 7 computed in 0.7416501045227051 seconds
row 8 computed in 0.7609500885009766 seconds
Done in 11.938772678375244


## Contents of tiny cython file

```python
import sage.libs.mpmath.all as mpmath

mpmath_ctx = mpmath.fp


def cykappa(r, u):
    return mpmath_ctx.besselk(1j*r, u)

def cykappa_with_mult(r, u):
    pi = mpmath_ctx.pi
    cdef double exp_Pih_R=mpmath_ctx.exp(pi * r / 2.)
    return mpmath_ctx.besselk(1j*r, u) * exp_Pih_R

def cyV(n, l, r, Q, zjlist, zjstarlist):
    pi = mpmath_ctx.pi
    cdef double exp_Pih_R=mpmath_ctx.exp(pi * r / 2.)
    cdef double complex ret = 0
    for zj, zjs in zip(zjlist, zjstarlist):
        xj = zj.real()
        xjs = zjs.real()
        yjs = zjs.imag()
        ret += mpmath_ctx.sqrt(yjs) * exp_Pih_R * cykappa(r, 2*pi*abs(l)*yjs) \
               * mpmath_ctx.exp(2*pi*1j*(l * xjs - n * xj))
    ret = ret / (2*Q)
    return ret
```

In [17]:
load("maass-cython.spyx")

Compiling ./maass-cython.spyx...


In [18]:
cykappa_with_mult(1, 3+i)

(0.06333777981976761-0.12791312356062806j)

In [19]:
kappa(1, 3+i)

0.0633377798197707 - 0.127913123560625*I

In [20]:
test_zj_starlist = make_zjstar_list(test_zj_list)

In [21]:
cyV(3, 1, 9.5**.5, 4, test_zj_list, test_zj_starlist)

(-0.0016458022789573614+0j)

In [22]:
V(3, 1, test_zj_list, 9.5**.5, 4)

-0.0016458022812606853 + 1.6263032587282567e-19*I

In [23]:
%time V(3, 1, test_zj_list, 9.5**.5, 4)

CPU times: user 37 ms, sys: 4.03 ms, total: 41 ms
Wall time: 40.1 ms


-0.0016458022812606853 + 1.6263032587282567e-19*I

In [24]:
%time cyV(3, 1, 9.5**.5, 4, test_zj_list, test_zj_starlist)

CPU times: user 3.11 ms, sys: 0 ns, total: 3.11 ms
Wall time: 3.12 ms


(-0.0016458022789573614+0j)

In [25]:
def make_matrix_with_some_cy(size, Y, r, withtime=True):
    if withtime:
        now = time.time()
    Q = size
    zjlist = make_zj_list(Y, Q)
    zjstarlist = make_zjstar_list(zjlist)
    if withtime:
        print("lists made in {}".format(time.time() - now))
    matrix = []
    B = []
    for n in range(-Q, Q+1):
        if withtime:
            now = time.time()
        if n == 0:
            continue
        row = []
        for l in range(-Q, Q+1):
            if l == 0:
                continue
            entry = cyV(n, l, CDF(r), CDF(Q), zjlist, zjstarlist=zjstarlist)
            if n == l:
                #entry = entry - sqrt(Y) * bessel_K(i*r, 2 * pi * abs(n) * Y)
                entry = entry - CDF(sqrt(Y)) * cykappa_with_mult(r, 2 * pi * abs(n) * Y)
            if l != 1:
                row.append(entry)
            else:
                B.append([-entry])
                #if n == 1:
                #    print(V(n, l, zjlist, r, Q, zjstarlist=zjstarlist))
                #    print(sqrt(Y) * kappa(r, 2 * pi * abs(n) * Y))
        matrix.append(row)
        if withtime:
            print("row {} computed in {} seconds".format(n, time.time() - now))
    return matrix, B

In [26]:
Q = 8
Y = 0.28
r = 9.53369526135
#testmat, testB = make_matrix(20, 0.5, 3)
now = time.time()
testmat, testB = make_matrix_with_some_cy(Q, Y, r)
print("Done in {}".format(time.time() - now))

lists made in 0.027595996856689453
row -8 computed in 0.06521344184875488 seconds
row -7 computed in 0.06373453140258789 seconds
row -6 computed in 0.06558513641357422 seconds
row -5 computed in 0.06500887870788574 seconds
row -4 computed in 0.06371355056762695 seconds
row -3 computed in 0.06500244140625 seconds
row -2 computed in 0.06799507141113281 seconds
row -1 computed in 0.06856870651245117 seconds
row 1 computed in 0.06412076950073242 seconds
row 2 computed in 0.06606030464172363 seconds
row 3 computed in 0.06769704818725586 seconds
row 4 computed in 0.06514859199523926 seconds
row 5 computed in 0.07109880447387695 seconds
row 6 computed in 0.06808328628540039 seconds
row 7 computed in 0.07031130790710449 seconds
row 8 computed in 0.06719779968261719 seconds
Done in 1.0943748950958252


In [43]:
def solve_for_coeffs(Q, Y, r):
    mat, B = make_matrix_with_some_cy(Q, Y, r, withtime=False)
    preinversiontime = time.time()
    mm = matrix(CC, mat[1:])
    Bm = matrix(CC, B[1:])
    res = mm.inverse() * Bm
    print("Inversion done in {}".format(time.time() - preinversiontime))
    indices = [i for i in range(-Q, Q) if i != 0 and i != 1]
    resdict = dict()
    for index, re in zip(indices, res):
        resdict[index] = re[0].real()
    return resdict

In [44]:
%time coeffdict = solve_for_coeffs(Q, Y, r)

Inversion done in 0.004590511322021484
CPU times: user 849 ms, sys: 2 µs, total: 849 ms
Wall time: 848 ms


In [29]:
coeffdict

{-8: 1.87322686737660e9,
 -7: 0.793986924762749,
 -6: -0.488696770726782,
 -5: 0.290607870282857,
 -4: -0.141331593018057,
 -3: 0.456186618172060,
 -2: 1.06833016591228,
 -1: -0.999999234922817,
 2: -1.06833082621984,
 3: -0.456187120225734,
 4: 0.141331341278230,
 5: -0.290607775330800,
 6: 0.488696650679484,
 7: -0.793986654374832}

In [30]:
def error_func(r, Y1=0.28, Y2=0.3, Q=8, power=2):
    coeffdict1 = solve_for_coeffs(Q, Y1, r)
    coeffdict2 = solve_for_coeffs(Q, Y2, r)
    return ((coeffdict1[2] - coeffdict2[2])**power
            + (coeffdict1[3] - coeffdict2[3]) **power
            + (coeffdict1[5] - coeffdict2[5])**power)

In [31]:
%time error_func(r)

CPU times: user 2.11 s, sys: 0 ns, total: 2.11 s
Wall time: 2.11 s


1.75925992991161e-9

In [32]:
def secant_method_single_iteration(r1, r2, Q=8, power=2):
    e1 = error_func(r1, Q=Q, power=power)
    e2 = error_func(r2, Q=Q, power=power)
    return r1 - e1*(r1 - r2)/(e1 - e2)

In [33]:
r1 = 9.5
r2 = 9.55
rlist = [r1, r2]
NOW = time.time()
now = time.time()
for idx in range(10):
    rnew = secant_method_single_iteration(rlist[-2], rlist[-1], power=2)
    rlist.append(rnew)
    print("computed r: {}      time: {}".format(rnew, time.time() - now))
    now = time.time()
print("Done in {}".format(time.time() - NOW))

computed r: 9.49727787715483      time: 4.218823432922363
computed r: 9.49419505507187      time: 4.247869253158569
computed r: 9.54086107605566      time: 4.4445717334747314
computed r: 9.58660052314911      time: 4.320803642272949
computed r: 9.53628031109461      time: 4.251846790313721
computed r: 9.53592508008126      time: 4.254331827163696
computed r: 9.53501930080858      time: 4.195786714553833
computed r: 9.53458670696755      time: 4.231893539428711
computed r: 9.53425301595749      time: 4.3261613845825195
computed r: 9.53404871373198      time: 4.3119964599609375
Done in 42.80536127090454


In [34]:
r1 = 9.5
r2 = 9.55
rlist = [r1, r2]
NOW = time.time()
now = time.time()
for idx in range(10):
    rnew = secant_method_single_iteration(rlist[-2], rlist[-1], power=1)
    rlist.append(rnew)
    print("computed r: {}      time: {}".format(rnew, time.time() - now))
    now = time.time()
print("Done in {}".format(time.time() - NOW))

computed r: 9.50928635143440      time: 4.294243335723877
computed r: 9.51594812823334      time: 4.298494815826416
computed r: 9.55109466860222      time: 4.213788747787476
computed r: 9.51984614699136      time: 4.181384563446045
computed r: 9.52287431424736      time: 4.1736743450164795
computed r: 9.53997579418115      time: 4.2550413608551025
computed r: 9.53071732375602      time: 4.253326416015625
computed r: 9.53286851162266      time: 4.2063398361206055
computed r: 9.53380225470783      time: 4.2565460205078125
computed r: 9.53369097168154      time: 4.279160499572754
Done in 42.413029193878174


In [35]:
r1 = 9.5
r2 = 9.55
rlist = [r1, r2]
NOW = time.time()
now = time.time()
for idx in range(10):
    rnew = secant_method_single_iteration(rlist[-2], rlist[-1], Q=6)
    rlist.append(rnew)
    print("computed r: {}      time: {}".format(rnew, time.time() - now))
    now = time.time()
print("Done in {}".format(time.time() - NOW))

computed r: 9.59387047826304      time: 2.0064644813537598
computed r: 9.54812422154999      time: 1.9785053730010986
computed r: 9.54664217265963      time: 1.9638545513153076
computed r: 9.54102948022800      time: 1.9975497722625732
computed r: 9.53857710063801      time: 1.9998939037322998
computed r: 9.53663247702626      time: 1.9896209239959717
computed r: 9.53543668711281      time: 1.9931871891021729
computed r: 9.53461087207954      time: 1.961972951889038
computed r: 9.53400807291181      time: 1.9911930561065674
computed r: 9.53340983480165      time: 2.0326008796691895
Done in 19.916143894195557


In [36]:
error_func(r, Q=6)

0.00120185116773810

I don't quite understand the required bounds on $Q$ at the moment. But surprisingly small values of $Q$ work surprisingly well.

In [37]:
r1 = 12.0
r2 = 12.5
rlist = [r1, r2]
NOW = time.time()
now = time.time()
for idx in range(20):
    rnew = secant_method_single_iteration(rlist[-2], rlist[-1], Q=8)
    rlist.append(rnew)
    print("computed r: {}      time: {}".format(rnew, time.time() - now))
    now = time.time()
print("Done in {}".format(time.time() - NOW))

computed r: 11.9729254449747      time: 4.913889408111572
computed r: 11.9416688434772      time: 4.794404029846191
computed r: 12.1450949287850      time: 4.843990087509155
computed r: 12.2230004572527      time: 4.822888374328613
computed r: 12.1422415428310      time: 4.694787979125977
computed r: 12.1388294940932      time: 4.670587539672852
computed r: 12.1645976731964      time: 4.507941246032715
computed r: 12.1675402709553      time: 4.502606153488159
computed r: 12.1699548228749      time: 4.490269660949707
computed r: 12.1711457765677      time: 4.542780876159668
computed r: 12.1718966760731      time: 4.654466152191162
computed r: 12.1723466561403      time: 4.651961326599121
computed r: 12.1726358055689      time: 4.6737775802612305
computed r: 12.1728356805994      time: 4.634793758392334
computed r: 12.1730106435920      time: 4.5980284214019775
computed r: 12.1733847007345      time: 4.576426982879639
computed r: 12.1728393846562      time: 4.645724296569824
computed r: 

In [38]:
r1 = 12.15
r2 = 12.2
rlist = [r1, r2]
NOW = time.time()
now = time.time()
for idx in range(10):
    rnew = secant_method_single_iteration(rlist[-2], rlist[-1], Q=6)
    rlist.append(rnew)
    print("computed r: {}      time: {}".format(rnew, time.time() - now))
    now = time.time()
print("Done in {}".format(time.time() - NOW))

computed r: 12.2686437411483      time: 2.2614376544952393
computed r: 12.1999812501054      time: 2.2313296794891357
computed r: 12.1999625422140      time: 2.287598133087158
computed r: 12.1925157918053      time: 2.2645509243011475
computed r: 12.1891964621255      time: 2.315746784210205
computed r: 12.1855309679207      time: 2.325157880783081
computed r: 12.1811475765694      time: 2.2331173419952393
computed r: 12.2202435647939      time: 2.2760608196258545
computed r: 12.1806731539924      time: 2.2943270206451416
computed r: 12.1801597939819      time: 2.242920398712158
Done in 22.73322367668152


I am rapidly running into precision errors. Many of these are caused, I think, by the larger-than-necessary K-bessel functions. I'm using $K_{ir}(u)$ instead of Andy's $\kappa_{ir}(u)$, simply because the libraries I have can compute these quickly. I should try to find a better K-bessel computation.

In [39]:
r1 = 12.15
r2 = 12.2
rlist = [r1, r2]
NOW = time.time()
now = time.time()
for idx in range(10):
    rnew = secant_method_single_iteration(rlist[-2], rlist[-1], Q=10)
    rlist.append(rnew)
    print("computed r: {}      time: {}".format(rnew, time.time() - now))
    now = time.time()
print("Done in {}".format(time.time() - NOW))

computed r: 12.1495446393491      time: 8.38785982131958
computed r: 12.1490742084784      time: 8.39201021194458
computed r: 12.1694270709510      time: 8.429820775985718
computed r: 12.1705659328399      time: 8.448359489440918
computed r: 12.1716801364197      time: 8.345372438430786
computed r: 12.1721914837349      time: 8.46572494506836
computed r: 12.1725174310769      time: 8.561171531677246
computed r: 12.1727071115659      time: 8.621055841445923
computed r: 12.1728235276695      time: 8.472947597503662
computed r: 12.1728944076246      time: 8.44180178642273
Done in 84.56763291358948


There are apparently precision problems; see the following computation. $Q = 20$ is much higher, but the actual convergence is not any faster.

In [40]:
r1 = 12.15
r2 = 12.2
rlist = [r1, r2]
NOW = time.time()
now = time.time()
for idx in range(10):
    rnew = secant_method_single_iteration(rlist[-2], rlist[-1], Q=20)
    rlist.append(rnew)
    print("computed r: {}      time: {}".format(rnew, time.time() - now))
    now = time.time()
print("Done in {}".format(time.time() - NOW))

computed r: 12.1013321704435      time: 54.434868574142456
computed r: 12.2206422540283      time: 51.76845192909241
computed r: 10.8987101117853      time: 52.83650493621826
computed r: 12.2228958502830      time: 52.07058835029602
computed r: 12.2257154533566      time: 50.5456326007843
computed r: 12.2147250575029      time: 51.26259922981262
computed r: 12.2089048844324      time: 52.52742004394531
computed r: 12.2000916828450      time: 52.26845932006836
computed r: 12.1927803450194      time: 51.81403970718384
computed r: 12.1863892009106      time: 50.97339653968811
Done in 520.5033602714539


Ah, or maybe everything is improved if we use the power 1 in the error function, instead of the power 2. I had thought finding a minimum would be superior to finding a zero, but not with the secant method setup I suppose.

In [41]:
r1 = 12.15
r2 = 12.2
rlist = [r1, r2]
NOW = time.time()
now = time.time()
for idx in range(10):
    rnew = secant_method_single_iteration(rlist[-2], rlist[-1], Q=20, power=1)
    rlist.append(rnew)
    print("computed r: {}      time: {}".format(rnew, time.time() - now))
    now = time.time()
print("Done in {}".format(time.time() - NOW))

computed r: 12.1741257896385      time: 50.750403881073
computed r: 12.1729719029430      time: 45.4972448348999
computed r: 12.1730084069161      time: 43.65937519073486
computed r: 12.1730083260559      time: 41.755942583084106
computed r: 12.1730083253282      time: 41.49747371673584
computed r: 12.1730083266425      time: 41.39926767349243
computed r: 12.1730083344389      time: 42.43319535255432
computed r: 12.1730083253559      time: 41.70858097076416
computed r: 12.1730083261962      time: 41.388752698898315
computed r: 12.1730083258166      time: 41.44952416419983
Done in 431.5410497188568
